# Analyse de texte

Le traitement du langage naturel (NLP) est une branche de l’intelligence artificielle (IA) qui traite du langage écrit et parlé. Vous pouvez utiliser le NLP pour créer des solutions qui extraient le sens sémantique du texte ou de la parole, ou qui formulent des réponses significatives en langage naturel.

Les *Cognitive Services* comprennent le service *Analyse de texte*, qui offre des fonctionnalités NLP prêtes à l’emploi, notamment l’identification de phrases clés dans le texte et la classification du texte en fonction du sentiment.

![Un robot lisant un carnet de notes](./images/NLP.jpg)

Par exemple, supposons que l’organisation fictive *Margie’s Travel* encourage ses clients à soumettre des avis sur leurs séjours à l’hôtel. Vous pourriez utiliser le service d’analyse de texte pour résumer les avis en extrayant des phrases clés, déterminer quels avis sont positifs et lesquels sont négatifs, ou analyser le texte des avis pour y trouver des mentions d’entités connues telles que des emplacements ou des personnes.

## Voir les documents d’avis

Commençons par jeter un coup d’œil à des avis d’hôtels laissés par des clients.

Ces avis sont dans des fichiers texte. Pour les voir, exécutez le code ci-dessous en cliquant sur le bouton **Exécuter la cellule** (&#9655;) à gauche de la cellule.

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Créer une ressource Cognitive Services

Pour analyser le texte de ces avis, vous pouvez utiliser le service cognitif **Analyse de texte**. Pour l’utiliser, vous devez provisionner soit une ressource **Analyse de texte** soit une ressource **Cognitive Services** dans votre abonnement Azure (utilisez une ressource Analyse de texte si c’est le seul service que vous prévoyez d’utiliser ou si vous voulez suivre son utilisation séparément ; sinon, vous pouvez utiliser une ressource Cognitive Services pour combiner le service Analyse de texte avec d’autres services cognitifs ; ce qui permet aux développeurs d’utiliser un seul point de terminaison et une seule clé pour y accéder).

Si vous n’en avez pas encore, suivez les étapes suivantes pour créer une ressource **Cognitive Services** dans votre abonnement Azure :

> **Remarque** : Si vous disposez déjà d’une ressource Cognitive Services, il suffit d’ouvrir sa page **Démarrage rapide** dans le portail Azure et de copier sa clé et son point de terminaison dans la cellule ci-dessous. Sinon, suivez les étapes ci-dessous pour en créer une.

1. Dans un autre onglet du navigateur, ouvrez le portail Azure à l’adresse https://portal.azure.com, en vous connectant avec votre compte Microsoft.
2. Cliquez sur le bouton **&#65291; Créer une ressource**, recherchez *Cognitive Services*, et créez une ressource **Cognitive Services** avec les paramètres suivants :
    - **Abonnement** : *Votre abonnement Azure*.
    - **Groupe de ressources** : *Sélectionnez ou créez un groupe de ressources portant un nom unique*.
    - **Région** : *Choisissez une région disponible* :
    - **Nom** : *Saisissez un nom unique*.
    - **Niveau tarifaire** : S0
    - **Je confirme avoir lu et compris les avis** : Sélectionné.
3. Attendez la fin du déploiement. Ensuite, accédez à votre ressource Cognitive Services et, sur la page **Aperçu**, cliquez sur le lien permettant de gérer les clés du service. Vous aurez besoin du point de terminaison et des clés pour vous connecter à votre ressource Cognitive Services à partir d’applications clientes.

### Obtenir la clé et le point de terminaison de votre ressource Cognitive Services

Pour utiliser votre ressource Cognitive Services, les applications clientes ont besoin de son point de terminaison et de sa clé d’authentification :

1. Dans le portail Azure, sur la page **Clés et Point de terminaison** de votre ressource Cognitive Services, copiez la **Clé 1** de votre ressource et collez-la dans le code ci-dessous, en remplaçant **YOUR_COG_KEY**.
2. Copiez le **point de terminaison** de votre ressource et collez-le dans le code ci-dessous, en remplaçant **YOUR_COG_ENDPOINT**.
3. Exécutez le code dans la cellule ci-dessous en cliquant sur son bouton vert <span style="color:green">&#9655;</span>.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Détecter la langue
Commençons par identifier la langue dans laquelle ces avis sont rédigés.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Extraire des phrases clés

Vous pouvez maintenant analyser le texte des avis des clients pour identifier des phrases clés qui donnent une idée des principaux points de discussion.

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Les phrases clés peuvent vous aider à comprendre les points de discussion les plus importants de chaque avis. Par exemple, un avis contenant la phrase « personnel serviable » ou « service médiocre » peut vous donner une indication des principales préoccupations de l’auteur de l’avis.

## Déterminer le sentiment

Il peut être utile de classer les avis comme *positifs* ou *négatifs* en fonction d’un *score de sentiment*. Là encore, vous pouvez utiliser le service Analyse de texte pour le faire.

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Extraire les entités connues

Les *entités* sont des éléments qui peuvent être mentionnés dans le texte et qui font référence à un type d’élément communément compris. Par exemple, un emplacement, une personne ou une date. Supposons que vous soyez intéressé par les dates et les lieux mentionnés dans les avis ; vous pouvez utiliser le code suivant pour les trouver.

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Notez que certaines entités sont suffisamment connues pour avoir une page Wikipédia associée, auquel cas le service Analyse de texte renvoie l’URL de cette page.

## En savoir plus

Pour plus d’informations sur le service Analyse de texte, consultez [la documentation sur le service Analyse de texte](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)